In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
import joblib

def generate_synthetic_data(num_samples=2000):
    # Generate synthetic dataset as before
    samples_per_category = num_samples // 3
    normal_spo2 = np.random.uniform(95, 100, samples_per_category)
    mild_spo2 = np.random.uniform(90, 94, samples_per_category)
    severe_spo2 = np.random.uniform(85, 89, samples_per_category)
    
    normal_hr = np.random.uniform(60, 100, samples_per_category)
    mild_hr = np.random.uniform(50, 110, samples_per_category)
    severe_hr = np.random.uniform(40, 120, samples_per_category)

    spo2 = np.concatenate([normal_spo2, mild_spo2, severe_spo2])
    heart_rate = np.concatenate([normal_hr, mild_hr, severe_hr])
    labels = np.array([0] * samples_per_category + [1] * samples_per_category + [2] * samples_per_category)

    # Shuffle the data
    indices = np.random.permutation(len(spo2))
    spo2_shuffled = spo2[indices]
    heart_rate_shuffled = heart_rate[indices]
    labels_shuffled = labels[indices]

    data = pd.DataFrame({
        "SpO2": spo2_shuffled,
        "HeartRate": heart_rate_shuffled,
        "Label": labels_shuffled
    })
    return data

# Generate dataset
data = generate_synthetic_data()

# Feature extraction
X = data[["SpO2", "HeartRate"]].values
y = data["Label"].values

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Define the CNN model
def build_model(input_shape, num_classes):
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),  
        layers.Reshape((input_shape[0], 1, 1)),  
        layers.Conv2D(32, (1, 1), activation="relu"),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(num_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# Model training
input_shape = (2,)  # 2 features: SpO2 and Heart Rate
num_classes = 3  # Normal, Mild Hypoxia, Severe Hypoxia
model = build_model(input_shape, num_classes)

# Calculate class weights to handle imbalanced classes
class_weights = {0: 1., 1: 2., 2: 1.5}

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32, class_weight=class_weights)

# Save the trained model
model.save('models/pulse_model.h5')

# Save the scaler
joblib.dump(scaler, 'models/scaler.pkl')

print("Model and scaler saved successfully.")


Epoch 1/50


C:\Users\CHERRY\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5862 - loss: 1.4449 - val_accuracy: 0.8750 - val_loss: 0.5161
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9029 - loss: 0.7158 - val_accuracy: 0.9525 - val_loss: 0.2524
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9706 - loss: 0.3294 - val_accuracy: 0.9900 - val_loss: 0.1297
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9987 - loss: 0.1690 - val_accuracy: 0.9950 - val_loss: 0.0753
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0891 - val_accuracy: 1.0000 - val_loss: 0.0446
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0565 - val_accuracy: 1.0000 - val_loss: 0.0327
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0430 - val_accuracy: 1.0000 - val_loss: 0.0231
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0276 - val_accuracy: 1.0000 - val_loss: 0.0170
Epo

Model and scaler saved successfully.
